# Importing Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model #GARCH Models
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
import math
import scipy.stats
from statistics import NormalDist
from scipy.stats import t
from scipy.stats import f
from datetime import datetime
from scipy.stats import shapiro #Shapiro Test
from scipy import stats #t-test
import statsmodels.api as sm #Ljung-Box Test
from scipy.stats import chi2

# Price and Returns

In [2]:
start_date = '2006-01-01' #yyyy-mm-dd
end_date = '2021-01-01'


# Vietnam
VT = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/2006-2021/HNX.csv')
VT['Date'] = pd.to_datetime(VT['Date'])
VT = VT.rename(columns={'Price': 'Close'})
VT = VT.replace(',','', regex=True)
VT['Close'] = VT['Close'].astype(float, errors = 'raise')
mask = (VT['Date'] >= start_date) & (VT['Date'] <= end_date)
VT = VT.loc[mask]
VT = VT.set_index('Date')
VT = VT.sort_index(axis=0, ascending = False)

In [3]:
def get_returns(df,d):
    '''
    The function obtains the log returns of the asset shifted d days
    
    PARAMETERS
    ----------
    df : pandas.DataFrame
        The data frame contains data of a chosen stock index 
        (Stock index must be arrange in DESCENDING ORDER by DATE)
    d : int
        The dth day being forecast
        (Assumed to be 1 for most cases)
        
    RETURNS
    -------
    df : pandas.DataFrame
        The data frame returns an updated data frame containing the
        'Returns' column
    '''
    df['Previous'] = df['Close'].shift(-d)
    df['Returns'] = np.log(df['Close']/df['Previous'])*100
    display(df)
    return df

VT = get_returns(VT, 1)

,Close,Open,High,Low,Vol.,Change %,Previous,Returns
Date,,,,,,,,
2020-12-31,203.12,196.94,203.15,194.34,100.18K,3.14%,196.94,3.089782
2020-12-30,196.94,197.10,197.32,192.06,110.79K,-0.08%,197.10,-0.081210
2020-12-29,197.10,196.58,197.85,194.20,101.70K,0.27%,196.57,0.269261
2020-12-28,196.57,192.46,198.08,192.44,122.33K,2.14%,192.46,2.113026
2020-12-25,192.46,187.63,192.67,183.87,98.21K,2.57%,187.63,2.541640
...,...,...,...,...,...,...,...,...
2006-01-13,90.55,90.55,90.84,90.55,0.32K,-0.26%,90.79,-0.264696
2006-01-11,90.79,90.79,90.79,90.79,0.24K,0.00%,90.79,0.000000
2006-01-09,90.79,90.79,90.79,90.79,0.08K,-0.15%,90.93,-0.154083


# Diebold-Mariano Test

In [4]:
# Author   : John Tsang
# Date     : December 7th, 2017
# Purpose  : Implement the Diebold-Mariano Test (DM test) to compare 
#            forecast accuracy
# Input    : 1) actual_lst: the list of actual values
#            2) pred1_lst : the first list of predicted values
#            3) pred2_lst : the second list of predicted values
#            4) h         : the number of stpes ahead
#            5) crit      : a string specifying the criterion 
#                             i)  MSE : the mean squared error
#                            ii)  MAD : the mean absolute deviation
#                           iii) MAPE : the mean absolute percentage error
#                            iv) poly : use power function to weigh the errors
#            6) poly      : the power for crit power 
#                           (it is only meaningful when crit is "poly")
# Condition: 1) length of actual_lst, pred1_lst and pred2_lst is equal
#            2) h must be an integer and it must be greater than 0 and less than 
#               the length of actual_lst.
#            3) crit must take the 4 values specified in Input
#            4) Each value of actual_lst, pred1_lst and pred2_lst must
#               be numerical values. Missing values will not be accepted.
#            5) power must be a numerical value.
# Return   : a named-tuple of 2 elements
#            1) p_value : the p-value of the DM test
#            2) DM      : the test statistics of the DM test
##########################################################
# References:
#
# Harvey, D., Leybourne, S., & Newbold, P. (1997). Testing the equality of 
#   prediction mean squared errors. International Journal of forecasting, 
#   13(2), 281-291.
#
# Diebold, F. X. and Mariano, R. S. (1995), Comparing predictive accuracy, 
#   Journal of business & economic statistics 13(3), 253-264.
#
##########################################################
def dm_test(actual_lst, pred1_lst, pred2_lst, h = 1, crit="MSE", power = 2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt,msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt,msg)
        len_act = len(actual_lst)
        len_p1  = len(pred1_lst)
        len_p2  = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt,msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt,msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt,msg)  
        # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")  
        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True
        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):  
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt,msg)
        return (rt,msg)
    
    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np
    
    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst  = []
    
    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()
    
    # Length of lists (as real numbers)
    T = float(len(actual_lst))
    
    # construct d according to crit
    if (crit == "MSE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append((actual - p1)**2)
            e2_lst.append((actual - p2)**2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(abs((actual - p1)/actual))
            e2_lst.append(abs((actual - p2)/actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual,p1,p2 in zip(actual_lst,pred1_lst,pred2_lst):
            e1_lst.append(((actual - p1))**(power))
            e2_lst.append(((actual - p2))**(power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)    
    
    # Mean of d        
    mean_d = pd.Series(d_lst).mean()
    
    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N-k):
              autoCov += ((Xi[i+k])-Xs)*(Xi[i]-Xs)
        return (1/(T))*autoCov
    gamma = []
    for lag in range(0,h):
        gamma.append(autocovariance(d_lst,len(d_lst),lag,mean_d)) # 0, 1, 2
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean_d
    harvey_adj=((T+1-2*h+h*(h-1)/T)/T)**(0.5)
    DM_stat = harvey_adj*DM_stat
    # Find p-value
    p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    
    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')
    
    rt = dm_return(DM = DM_stat, p_value = p_value)
    
    return rt

## Custom DM-Test Function

In [5]:
def DM_test(country, model_1, model_2, t):
    
    arr_model_1_dropped = model_1.drop(['Date'], axis = 1)
    arr_model_1_zero = arr_model_1_dropped.iloc[:, 0].to_numpy()

    arr_model_2_dropped = model_2.drop(['Date'], axis = 1)
    arr_model_2_zero = arr_model_2_dropped.iloc[:,0].to_numpy()

    returns = np.array(country['Returns'])[:t]

    dm = dm_test(returns, arr_model_1_zero, arr_model_2_zero, h = 1, crit = 'MSE')
    
    return dm

# Fixed Window

## 0.01%

In [6]:
VT_fw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/VT_VaR_FW-DN-0.01%25.csv')
VT_fw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/VT_VaR_FW-HS-0.01%25.csv')
VT_fw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/VT_VaR_FW-BRW-0.01%25.csv')
VT_fw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/VT_VaR_FW-G-0.01%25.csv')
VT_fw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/VT_VaR_FW-EA-0.01%25.csv')
VT_fw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/VT_VaR_FW-E-0.01%25.csv')
VT_fw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-FW/VT_fw_FW-I-0.01%25.csv')
VT_fw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-FW/VT_fw_FW-FI-0.01%25.csv')
VT_fw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-FW/VT_fw_FW-FI0.633-0.01%25.csv')

In [7]:
# DN
print(DM_test(VT, VT_fw_01_dn, VT_fw_01_garch, len(VT_fw_01_dn)))
print(DM_test(VT, VT_fw_01_dn, VT_fw_01_earch, len(VT_fw_01_dn)))
print(DM_test(VT, VT_fw_01_dn, VT_fw_01_egarch, len(VT_fw_01_dn)))
print(DM_test(VT, VT_fw_01_dn, VT_fw_01_igarch, len(VT_fw_01_dn)))
print(DM_test(VT, VT_fw_01_dn, VT_fw_01_figarch, len(VT_fw_01_dn)))
print(DM_test(VT, VT_fw_01_dn, VT_fw_01_figarch0633, len(VT_fw_01_dn)))

dm_return(DM=11.386282519119211, p_value=6.4180273452696245e-28)
dm_return(DM=9.169638376686807, p_value=4.035930714775796e-19)
dm_return(DM=15.149403302058642, p_value=1.0566276326193046e-45)
dm_return(DM=7.832060001145441, p_value=1.545733357090145e-14)
dm_return(DM=13.806273804519277, p_value=5.339666110029534e-39)
dm_return(DM=12.1494874929468, p_value=2.9205648285163882e-31)


In [8]:
# HS
print(DM_test(VT, VT_fw_01_hs, VT_fw_01_garch, len(VT_fw_01_hs)))
print(DM_test(VT, VT_fw_01_hs, VT_fw_01_earch, len(VT_fw_01_hs)))
print(DM_test(VT, VT_fw_01_hs, VT_fw_01_egarch, len(VT_fw_01_hs)))
print(DM_test(VT, VT_fw_01_hs, VT_fw_01_igarch, len(VT_fw_01_hs)))
print(DM_test(VT, VT_fw_01_hs, VT_fw_01_figarch, len(VT_fw_01_hs)))
print(DM_test(VT, VT_fw_01_hs, VT_fw_01_figarch0633, len(VT_fw_01_hs)))

dm_return(DM=27.921205295164068, p_value=6.778484616511082e-120)
dm_return(DM=36.521177918419184, p_value=8.068391330204634e-172)
dm_return(DM=32.12645203392422, p_value=1.628672625674056e-145)
dm_return(DM=20.253014277173783, p_value=8.806367965756964e-74)
dm_return(DM=30.368633962020088, p_value=7.765203210495023e-135)
dm_return(DM=28.038630259903066, p_value=1.2988072058029338e-120)


In [9]:
# BRW
print(DM_test(VT, VT_fw_01_brw, VT_fw_01_garch, len(VT_fw_01_brw)))
print(DM_test(VT, VT_fw_01_brw, VT_fw_01_earch, len(VT_fw_01_brw)))
print(DM_test(VT, VT_fw_01_brw, VT_fw_01_egarch, len(VT_fw_01_brw)))
print(DM_test(VT, VT_fw_01_brw, VT_fw_01_igarch, len(VT_fw_01_brw)))
print(DM_test(VT, VT_fw_01_brw, VT_fw_01_figarch, len(VT_fw_01_brw)))
print(DM_test(VT, VT_fw_01_brw, VT_fw_01_figarch0633, len(VT_fw_01_brw)))

dm_return(DM=0.606923239524968, p_value=0.5440762374944228)
dm_return(DM=-10.581442231850536, p_value=1.46104339483987e-24)
dm_return(DM=2.970838937988596, p_value=0.00305990725721682)
dm_return(DM=1.0286459462286295, p_value=0.3039609130838612)
dm_return(DM=2.4763538161665166, p_value=0.01348161795562758)
dm_return(DM=1.9417160704303824, p_value=0.052526677526706214)


In [10]:
# GARCH
print(DM_test(VT, VT_fw_01_garch, VT_fw_01_earch, len(VT_fw_01_garch)))
print(DM_test(VT, VT_fw_01_garch, VT_fw_01_egarch, len(VT_fw_01_garch)))
print(DM_test(VT, VT_fw_01_garch, VT_fw_01_igarch, len(VT_fw_01_garch)))
print(DM_test(VT, VT_fw_01_garch, VT_fw_01_figarch, len(VT_fw_01_garch)))
print(DM_test(VT, VT_fw_01_garch, VT_fw_01_figarch0633, len(VT_fw_01_garch)))

dm_return(DM=-7.400734266069281, p_value=3.4699980729302404e-13)
dm_return(DM=8.882650079557, p_value=4.344552192633267e-18)
dm_return(DM=1.0063903924114335, p_value=0.3145359585172153)
dm_return(DM=10.231187610102102, p_value=3.709682559014881e-23)
dm_return(DM=5.031293687441708, p_value=6.03483842401593e-07)


In [11]:
# EARCH
print(DM_test(VT, VT_fw_01_earch, VT_fw_01_egarch, len(VT_fw_01_earch)))
print(DM_test(VT, VT_fw_01_earch, VT_fw_01_igarch, len(VT_fw_01_earch)))
print(DM_test(VT, VT_fw_01_earch, VT_fw_01_figarch, len(VT_fw_01_earch)))
print(DM_test(VT, VT_fw_01_earch, VT_fw_01_figarch0633, len(VT_fw_01_earch)))

dm_return(DM=10.791760673712588, p_value=2.0161256365220118e-25)
dm_return(DM=6.217587421304666, p_value=8.172548147166987e-10)
dm_return(DM=10.376645913674869, p_value=9.777860648067721e-24)
dm_return(DM=9.541665433366513, p_value=1.698241992669119e-20)


In [12]:
# EGARCH
print(DM_test(VT, VT_fw_01_egarch, VT_fw_01_igarch, len(VT_fw_01_egarch)))
print(DM_test(VT, VT_fw_01_egarch, VT_fw_01_figarch, len(VT_fw_01_egarch)))
print(DM_test(VT, VT_fw_01_egarch, VT_fw_01_figarch0633, len(VT_fw_01_egarch)))

dm_return(DM=-0.6296590333161388, p_value=0.5290997453708288)
dm_return(DM=-1.1017851071041118, p_value=0.2708906641304711)
dm_return(DM=-1.229263705199604, p_value=0.21933895277810816)


In [13]:
# IGARCH
print(DM_test(VT, VT_fw_01_igarch, VT_fw_01_figarch, len(VT_fw_01_igarch)))
print(DM_test(VT, VT_fw_01_igarch, VT_fw_01_figarch0633, len(VT_fw_01_igarch)))

dm_return(DM=0.5217726193406105, p_value=0.6019749076045198)
dm_return(DM=0.3617645097602595, p_value=0.7176247535766735)


In [14]:
# FIGARCH
print(DM_test(VT, VT_fw_01_figarch, VT_fw_01_figarch0633, len(VT_fw_01_figarch)))

dm_return(DM=-1.1817711567330318, p_value=0.23765211955410756)


## 1.00 %

In [15]:
VT_fw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/VT_VaR_FW-DN-1%25.csv')
VT_fw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/VT_VaR_FW-HS-1%25.csv')
VT_fw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/VT_VaR_FW-BRW-1%25.csv')
VT_fw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/VT_VaR_FW-G-1%25.csv')
VT_fw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/VT_VaR_FW-EA-1%25.csv')
VT_fw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/VT_VaR_FW-E-1%25.csv')
VT_fw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-FW/VT_fw_FW-I-1%25.csv')
VT_fw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-FW/VT_fw_FW-FI-1%25.csv')
VT_fw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-FW/VT_fw_FW-FI0.633-1%25.csv')

In [16]:
# DN
print(DM_test(VT, VT_fw_1_dn, VT_fw_1_garch, len(VT_fw_1_dn)))
print(DM_test(VT, VT_fw_1_dn, VT_fw_1_earch, len(VT_fw_1_dn)))
print(DM_test(VT, VT_fw_1_dn, VT_fw_1_egarch, len(VT_fw_1_dn)))
print(DM_test(VT, VT_fw_1_dn, VT_fw_1_igarch, len(VT_fw_1_dn)))
print(DM_test(VT, VT_fw_1_dn, VT_fw_1_figarch, len(VT_fw_1_dn)))
print(DM_test(VT, VT_fw_1_dn, VT_fw_1_figarch0633, len(VT_fw_1_dn)))

dm_return(DM=10.75459340034554, p_value=2.866948189834202e-25)
dm_return(DM=8.875594697023722, p_value=4.602443517882155e-18)
dm_return(DM=14.163947849311013, p_value=9.494483166062219e-41)
dm_return(DM=7.461062762748269, p_value=2.2655744837750662e-13)
dm_return(DM=12.95144351766396, p_value=6.290105624201765e-35)
dm_return(DM=11.452064827497086, p_value=3.3517940677518054e-28)


In [17]:
# HS
print(DM_test(VT, VT_fw_1_hs, VT_fw_1_garch, len(VT_fw_1_hs)))
print(DM_test(VT, VT_fw_1_hs, VT_fw_1_earch, len(VT_fw_1_hs)))
print(DM_test(VT, VT_fw_1_hs, VT_fw_1_egarch, len(VT_fw_1_hs)))
print(DM_test(VT, VT_fw_1_hs, VT_fw_1_igarch, len(VT_fw_1_hs)))
print(DM_test(VT, VT_fw_1_hs, VT_fw_1_figarch, len(VT_fw_1_hs)))
print(DM_test(VT, VT_fw_1_hs, VT_fw_1_figarch0633, len(VT_fw_1_hs)))

dm_return(DM=25.49624181524077, p_value=4.24233639859715e-105)
dm_return(DM=34.36675392612697, p_value=5.234793166610124e-159)
dm_return(DM=29.795209311554665, p_value=2.4305168421990615e-131)
dm_return(DM=17.81259382350667, p_value=6.349316131655293e-60)
dm_return(DM=27.99748189727568, p_value=2.3174177902860007e-120)
dm_return(DM=25.491722436135436, p_value=4.519596314464395e-105)


In [18]:
# BRW
print(DM_test(VT, VT_fw_1_brw, VT_fw_1_garch, len(VT_fw_1_brw)))
print(DM_test(VT, VT_fw_1_brw, VT_fw_1_earch, len(VT_fw_1_brw)))
print(DM_test(VT, VT_fw_1_brw, VT_fw_1_egarch, len(VT_fw_1_brw)))
print(DM_test(VT, VT_fw_1_brw, VT_fw_1_igarch, len(VT_fw_1_brw)))
print(DM_test(VT, VT_fw_1_brw, VT_fw_1_figarch, len(VT_fw_1_brw)))
print(DM_test(VT, VT_fw_1_brw, VT_fw_1_figarch0633, len(VT_fw_1_brw)))

dm_return(DM=2.348485283111692, p_value=0.01909555076438217)
dm_return(DM=-6.188605966458887, p_value=9.741988398956607e-10)
dm_return(DM=5.335919699618965, p_value=1.2427732568028544e-07)
dm_return(DM=1.9131419251060071, p_value=0.0560911041162968)
dm_return(DM=4.5408846636757385, p_value=6.475379744896128e-06)
dm_return(DM=3.6153336532759637, p_value=0.000318983521343847)


In [19]:
# GARCH
print(DM_test(VT, VT_fw_1_garch, VT_fw_1_earch, len(VT_fw_1_garch)))
print(DM_test(VT, VT_fw_1_garch, VT_fw_1_egarch, len(VT_fw_1_garch)))
print(DM_test(VT, VT_fw_1_garch, VT_fw_1_igarch, len(VT_fw_1_garch)))
print(DM_test(VT, VT_fw_1_garch, VT_fw_1_figarch, len(VT_fw_1_garch)))
print(DM_test(VT, VT_fw_1_garch, VT_fw_1_figarch0633, len(VT_fw_1_garch)))

dm_return(DM=-7.039830625256435, p_value=4.18346870782046e-12)
dm_return(DM=8.860289651916824, p_value=5.21507000032615e-18)
dm_return(DM=0.9336891607505258, p_value=0.3507495399674876)
dm_return(DM=9.948136542309719, p_value=4.770848441493265e-22)
dm_return(DM=4.875192457944327, p_value=1.3142767089705145e-06)


In [20]:
# EARCH
print(DM_test(VT, VT_fw_1_earch, VT_fw_1_egarch, len(VT_fw_1_earch)))
print(DM_test(VT, VT_fw_1_earch, VT_fw_1_igarch, len(VT_fw_1_earch)))
print(DM_test(VT, VT_fw_1_earch, VT_fw_1_figarch, len(VT_fw_1_earch)))
print(DM_test(VT, VT_fw_1_earch, VT_fw_1_figarch0633, len(VT_fw_1_earch)))

dm_return(DM=10.231640973643785, p_value=3.69437787622055e-23)
dm_return(DM=5.900972513665308, p_value=5.359265390484026e-09)
dm_return(DM=9.809428077427441, p_value=1.6352035110213898e-21)
dm_return(DM=9.018160212577467, p_value=1.4252304984298246e-18)


In [21]:
# EGARCH
print(DM_test(VT, VT_fw_1_egarch, VT_fw_1_igarch, len(VT_fw_1_egarch)))
print(DM_test(VT, VT_fw_1_egarch, VT_fw_1_figarch, len(VT_fw_1_egarch)))
print(DM_test(VT, VT_fw_1_egarch, VT_fw_1_figarch0633, len(VT_fw_1_egarch)))

dm_return(DM=-0.6894194850239146, p_value=0.49076176842365304)
dm_return(DM=-1.2552113979702504, p_value=0.20977319329833155)
dm_return(DM=-1.305751129311221, p_value=0.19201730736007447)


In [22]:
# IGARCH
print(DM_test(VT, VT_fw_1_igarch, VT_fw_1_figarch, len(VT_fw_1_igarch)))
print(DM_test(VT, VT_fw_1_igarch, VT_fw_1_figarch0633, len(VT_fw_1_igarch)))

dm_return(DM=0.56208725278882, p_value=0.5742160128865836)
dm_return(DM=0.4133335587912555, p_value=0.679474415582292)


In [23]:
# FIGARCH
print(DM_test(VT, VT_fw_1_figarch, VT_fw_1_figarch0633, len(VT_fw_1_figarch)))

dm_return(DM=-1.1731743539556163, p_value=0.2410794908128007)


## 5.00 %

In [24]:
VT_fw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/VT_VaR_FW-DN-5%25.csv')
VT_fw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/VT_VaR_FW-HS-5%25.csv')
VT_fw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/VT_VaR_FW-BRW-5%25.csv')
VT_fw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/VT_VaR_FW-G-5%25.csv')
VT_fw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/VT_VaR_FW-EA-5%25.csv')
VT_fw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/VT_VaR_FW-E-5%25.csv')
VT_fw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-FW/VT_fw_FW-I-5%25.csv')
VT_fw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-FW/VT_fw_FW-FI-5%25.csv')
VT_fw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-FW/VT_fw_FW-FI0.633-5%25.csv')

In [25]:
# DN
print(DM_test(VT, VT_fw_5_dn, VT_fw_5_garch, len(VT_fw_5_dn)))
print(DM_test(VT, VT_fw_5_dn, VT_fw_5_earch, len(VT_fw_5_dn)))
print(DM_test(VT, VT_fw_5_dn, VT_fw_5_egarch, len(VT_fw_5_dn)))
print(DM_test(VT, VT_fw_5_dn, VT_fw_5_igarch, len(VT_fw_5_dn)))
print(DM_test(VT, VT_fw_5_dn, VT_fw_5_figarch, len(VT_fw_5_dn)))
print(DM_test(VT, VT_fw_5_dn, VT_fw_5_figarch0633, len(VT_fw_5_dn)))

dm_return(DM=9.919444741510604, p_value=6.162025771723677e-22)
dm_return(DM=8.485904227674546, p_value=1.0506395361958691e-16)
dm_return(DM=12.890789904572232, p_value=1.2058623735746227e-34)
dm_return(DM=6.99590268843642, p_value=5.623705951024226e-12)
dm_return(DM=11.848811716978137, p_value=6.307070621001843e-30)
dm_return(DM=10.548342303814323, p_value=1.990225954979074e-24)


In [26]:
# HS
print(DM_test(VT, VT_fw_5_hs, VT_fw_5_garch, len(VT_fw_5_hs)))
print(DM_test(VT, VT_fw_5_hs, VT_fw_5_earch, len(VT_fw_5_hs)))
print(DM_test(VT, VT_fw_5_hs, VT_fw_5_egarch, len(VT_fw_5_hs)))
print(DM_test(VT, VT_fw_5_hs, VT_fw_5_igarch, len(VT_fw_5_hs)))
print(DM_test(VT, VT_fw_5_hs, VT_fw_5_figarch, len(VT_fw_5_hs)))
print(DM_test(VT, VT_fw_5_hs, VT_fw_5_figarch0633, len(VT_fw_5_hs)))

dm_return(DM=7.524761537473213, p_value=1.4398659373418944e-13)
dm_return(DM=3.6898824122205047, p_value=0.0002397476615189289)
dm_return(DM=10.3145242538747, p_value=1.7310322452442835e-23)
dm_return(DM=5.378020397646544, p_value=9.926325006059136e-08)
dm_return(DM=9.389005241301327, p_value=6.306179989992854e-20)
dm_return(DM=8.280866115769019, p_value=5.2053073613943e-16)


In [27]:
# BRW
print(DM_test(VT, VT_fw_5_brw, VT_fw_5_garch, len(VT_fw_5_brw)))
print(DM_test(VT, VT_fw_5_brw, VT_fw_5_earch, len(VT_fw_5_brw)))
print(DM_test(VT, VT_fw_5_brw, VT_fw_5_egarch, len(VT_fw_5_brw)))
print(DM_test(VT, VT_fw_5_brw, VT_fw_5_igarch, len(VT_fw_5_brw)))
print(DM_test(VT, VT_fw_5_brw, VT_fw_5_figarch, len(VT_fw_5_brw)))
print(DM_test(VT, VT_fw_5_brw, VT_fw_5_figarch0633, len(VT_fw_5_brw)))

dm_return(DM=-3.9911382095096375, p_value=7.187814470435362e-05)
dm_return(DM=-12.880203326829518, p_value=1.350650757812324e-34)
dm_return(DM=-2.3727942332765046, p_value=0.01789256700970567)
dm_return(DM=-1.6135763936212564, p_value=0.10701868230388877)
dm_return(DM=-2.4556921303394397, p_value=0.014275704535013378)
dm_return(DM=-2.324026302719262, p_value=0.020376877172028525)


In [28]:
# GARCH
print(DM_test(VT, VT_fw_5_garch, VT_fw_5_earch, len(VT_fw_5_garch)))
print(DM_test(VT, VT_fw_5_garch, VT_fw_5_egarch, len(VT_fw_5_garch)))
print(DM_test(VT, VT_fw_5_garch, VT_fw_5_igarch, len(VT_fw_5_garch)))
print(DM_test(VT, VT_fw_5_garch, VT_fw_5_figarch, len(VT_fw_5_garch)))
print(DM_test(VT, VT_fw_5_garch, VT_fw_5_figarch0633, len(VT_fw_5_garch)))

dm_return(DM=-6.552070820077864, p_value=1.0227469992971375e-10)
dm_return(DM=8.762007360983496, p_value=1.1587522008415797e-17)
dm_return(DM=0.8522345059758031, p_value=0.39434222964962773)
dm_return(DM=9.523271320088789, p_value=1.9908018700449194e-20)
dm_return(DM=4.664591939073263, p_value=3.63058559868103e-06)


In [29]:
# EARCH
print(DM_test(VT, VT_fw_5_earch, VT_fw_5_egarch, len(VT_fw_5_earch)))
print(DM_test(VT, VT_fw_5_earch, VT_fw_5_igarch, len(VT_fw_5_earch)))
print(DM_test(VT, VT_fw_5_earch, VT_fw_5_figarch, len(VT_fw_5_earch)))
print(DM_test(VT, VT_fw_5_earch, VT_fw_5_figarch0633, len(VT_fw_5_earch)))

dm_return(DM=9.47154783624229, p_value=3.1087137086849884e-20)
dm_return(DM=5.510246817293792, p_value=4.851889455419935e-08)
dm_return(DM=9.053234636069735, p_value=1.065738614150974e-18)
dm_return(DM=8.326796791511192, p_value=3.647194013562767e-16)


In [30]:
# EGARCH
print(DM_test(VT, VT_fw_5_egarch, VT_fw_5_igarch, len(VT_fw_5_egarch)))
print(DM_test(VT, VT_fw_5_egarch, VT_fw_5_figarch, len(VT_fw_5_egarch)))
print(DM_test(VT, VT_fw_5_egarch, VT_fw_5_figarch0633, len(VT_fw_5_egarch)))

dm_return(DM=-0.7489955608891944, p_value=0.45408282991791493)
dm_return(DM=-1.4078542320297474, p_value=0.15956760451787208)
dm_return(DM=-1.3799634132562157, p_value=0.16798831287824897)


In [31]:
# IGARCH
print(DM_test(VT, VT_fw_5_igarch, VT_fw_5_figarch,  len(VT_fw_5_igarch)))
print(DM_test(VT, VT_fw_5_igarch, VT_fw_5_figarch0633, len(VT_fw_5_igarch)))

dm_return(DM=0.6014479986986909, p_value=0.5477141815455198)
dm_return(DM=0.46532772211253365, p_value=0.6418250281326681)


In [32]:
# FIGARCH
print(DM_test(VT, VT_fw_5_figarch, VT_fw_5_figarch0633, len(VT_fw_5_figarch)))

dm_return(DM=-1.1561493179539812, p_value=0.24796959853009984)


# Rolling Window

## 0.01 %

In [33]:
VT_rw_01_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/0.01%25/VT_VaR_RW-DN-0.01%25.csv')
VT_rw_01_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/0.01%25/VT_VaR_RW-HS-0.01%25.csv')
VT_rw_01_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/0.01%25/VT_VaR_RW-BRW-0.01%25.csv')
VT_rw_01_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/0.01%25/VT_VaR_RW-G-0.01%25.csv')
VT_rw_01_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/0.01%25/VT_VaR_RW-EA-0.01%25.csv')
VT_rw_01_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/0.01%25/VT_VaR_RW-E-0.01%25.csv')
VT_rw_01_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-0.01%25-RW/VT_RW-I-0.01%25.csv')
VT_rw_01_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-0.01%25-RW/VT_RW-FI-0.01%25.csv')
VT_rw_01_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-0.01%25-RW/VT_RW-FI0.633-0.01%25.csv')

In [34]:
# DN
print(DM_test(VT, VT_rw_01_dn, VT_rw_01_garch, len(VT_rw_01_dn)))
print(DM_test(VT, VT_rw_01_dn, VT_rw_01_earch, len(VT_rw_01_dn)))
print(DM_test(VT, VT_rw_01_dn, VT_rw_01_egarch, len(VT_rw_01_dn)))
print(DM_test(VT, VT_rw_01_dn, VT_rw_01_igarch, len(VT_rw_01_dn)))
print(DM_test(VT, VT_rw_01_dn, VT_rw_01_figarch, len(VT_rw_01_dn)))
print(DM_test(VT, VT_rw_01_dn, VT_rw_01_figarch0633, len(VT_rw_01_dn)))

dm_return(DM=16.779184469203603, p_value=2.824360525077881e-54)
dm_return(DM=11.3163134263466, p_value=1.2771042430616482e-27)
dm_return(DM=19.461741292058907, p_value=3.2197143634931155e-69)
dm_return(DM=11.791784422612904, p_value=1.1228800177310145e-29)
dm_return(DM=19.674573407558583, p_value=1.9338685068446135e-70)
dm_return(DM=18.16602639089467, p_value=6.885360055997089e-62)


In [35]:
# HS
print(DM_test(VT, VT_rw_01_hs, VT_rw_01_garch, len(VT_rw_01_hs)))
print(DM_test(VT, VT_rw_01_hs, VT_rw_01_earch, len(VT_rw_01_hs)))
print(DM_test(VT, VT_rw_01_hs, VT_rw_01_egarch, len(VT_rw_01_hs)))
print(DM_test(VT, VT_rw_01_hs, VT_rw_01_igarch, len(VT_rw_01_hs)))
print(DM_test(VT, VT_rw_01_hs, VT_rw_01_figarch, len(VT_rw_01_hs)))
print(DM_test(VT, VT_rw_01_hs, VT_rw_01_figarch0633, len(VT_rw_01_hs)))

dm_return(DM=53.61409313572874, p_value=1.9222410739553993e-265)
dm_return(DM=65.86700614886485, p_value=4e-323)
dm_return(DM=59.63771648452544, p_value=9.721754702090161e-295)
dm_return(DM=36.73448040872456, p_value=4.449787593191157e-173)
dm_return(DM=58.596263661740174, p_value=8.298438161881757e-290)
dm_return(DM=54.69158789265401, p_value=8.018576935921988e-271)


In [36]:
# BRW
print(DM_test(VT, VT_rw_01_brw, VT_rw_01_garch, len(VT_rw_01_brw)))
print(DM_test(VT, VT_rw_01_brw, VT_rw_01_earch, len(VT_rw_01_brw)))
print(DM_test(VT, VT_rw_01_brw, VT_rw_01_egarch, len(VT_rw_01_brw)))
print(DM_test(VT, VT_rw_01_brw, VT_rw_01_igarch, len(VT_rw_01_brw)))
print(DM_test(VT, VT_rw_01_brw, VT_rw_01_figarch, len(VT_rw_01_brw)))
print(DM_test(VT, VT_rw_01_brw, VT_rw_01_figarch0633, len(VT_rw_01_brw)))

dm_return(DM=-0.09657813859028354, p_value=0.9230859127340207)
dm_return(DM=-14.409110197864715, p_value=5.791865371333201e-42)
dm_return(DM=0.699044308939691, p_value=0.4847300897873543)
dm_return(DM=0.7596930952360987, p_value=0.447664691294024)
dm_return(DM=1.6282637082291302, p_value=0.10386768304484575)
dm_return(DM=1.4020455272792116, p_value=0.16129444303016294)


In [37]:
# GARCH
print(DM_test(VT, VT_rw_01_garch, VT_rw_01_earch, len(VT_rw_01_garch)))
print(DM_test(VT, VT_rw_01_garch, VT_rw_01_egarch, len(VT_rw_01_garch)))
print(DM_test(VT, VT_rw_01_garch, VT_rw_01_igarch, len(VT_rw_01_garch)))
print(DM_test(VT, VT_rw_01_garch, VT_rw_01_figarch, len(VT_rw_01_garch)))
print(DM_test(VT, VT_rw_01_garch, VT_rw_01_figarch0633, len(VT_rw_01_garch)))

dm_return(DM=-10.869786353346178, p_value=9.600263527580288e-26)
dm_return(DM=4.281050647572782, p_value=2.0887681606624937e-05)
dm_return(DM=1.3444934044613854, p_value=0.1791747989370666)
dm_return(DM=9.033547836789763, p_value=1.2547349783463563e-18)
dm_return(DM=6.3747019214403835, p_value=3.1149001122529496e-10)


In [38]:
# EARCH
print(DM_test(VT, VT_rw_01_earch, VT_rw_01_egarch, len(VT_rw_01_earch)))
print(DM_test(VT, VT_rw_01_earch, VT_rw_01_igarch, len(VT_rw_01_earch)))
print(DM_test(VT, VT_rw_01_earch, VT_rw_01_figarch, len(VT_rw_01_earch)))
print(DM_test(VT, VT_rw_01_earch, VT_rw_01_figarch0633, len(VT_rw_01_earch)))

dm_return(DM=12.754391219536569, p_value=5.173524070105414e-34)
dm_return(DM=9.894236762614922, p_value=7.711819685122003e-22)
dm_return(DM=14.761098846000193, p_value=9.956726561625196e-44)
dm_return(DM=14.236185039160173, p_value=4.17668291500512e-41)


In [39]:
# EGARCH
print(DM_test(VT, VT_rw_01_egarch, VT_rw_01_igarch, len(VT_rw_01_egarch)))
print(DM_test(VT, VT_rw_01_egarch, VT_rw_01_figarch, len(VT_rw_01_egarch)))
print(DM_test(VT, VT_rw_01_egarch, VT_rw_01_figarch0633, len(VT_rw_01_egarch)))

dm_return(DM=0.5666413885475633, p_value=0.5711187945845249)
dm_return(DM=4.7477933646625035, p_value=2.4415505331507774e-06)
dm_return(DM=2.8551163198030647, p_value=0.004414837616247452)


In [40]:
# IGARCH
print(DM_test(VT, VT_rw_01_igarch, VT_rw_01_figarch,  len(VT_rw_01_igarch)))
print(DM_test(VT, VT_rw_01_igarch, VT_rw_01_figarch0633, len(VT_rw_01_igarch)))

dm_return(DM=0.3634306842054101, p_value=0.7163804125510207)
dm_return(DM=0.28375094953954155, p_value=0.7766755937646204)


In [41]:
# FIGARCH
print(DM_test(VT, VT_rw_01_figarch, VT_rw_01_figarch0633, len(VT_rw_01_figarch)))

dm_return(DM=-0.8893255172290961, p_value=0.37409887962765886)


## 1.00 %

In [42]:
VT_rw_1_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/1%25/VT_VaR_RW-DN-1%25.csv')
VT_rw_1_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/1%25/VT_VaR_RW-HS-1%25.csv')
VT_rw_1_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/1%25/VT_VaR_RW-BRW-1%25.csv')
VT_rw_1_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/1%25/VT_VaR_RW-G-1%25.csv')
VT_rw_1_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/1%25/VT_VaR_RW-EA-1%25.csv')
VT_rw_1_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/1%25/VT_VaR_RW-E-1%25.csv')
VT_rw_1_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-1%25-RW/VT_RW-I-1%25.csv')
VT_rw_1_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-1%25-RW/VT_RW-FI-1%25.csv')
VT_rw_1_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-1%25-RW/VT_RW-FI0.633-1%25.csv')

In [43]:
# DN
print(DM_test(VT, VT_rw_1_dn, VT_rw_1_garch, len(VT_rw_1_dn)))
print(DM_test(VT, VT_rw_1_dn, VT_rw_1_earch, len(VT_rw_1_dn)))
print(DM_test(VT, VT_rw_1_dn, VT_rw_1_egarch, len(VT_rw_1_dn)))
print(DM_test(VT, VT_rw_1_dn, VT_rw_1_igarch, len(VT_rw_1_dn)))
print(DM_test(VT, VT_rw_1_dn, VT_rw_1_figarch, len(VT_rw_1_dn)))
print(DM_test(VT, VT_rw_1_dn, VT_rw_1_figarch0633, len(VT_rw_1_dn)))

dm_return(DM=15.77782890364882, p_value=5.902002875835977e-49)
dm_return(DM=10.967191579796891, p_value=3.781466694001127e-26)
dm_return(DM=18.145961696039247, p_value=8.910326544020621e-62)
dm_return(DM=11.184582331356102, p_value=4.626865072069354e-27)
dm_return(DM=18.33305506009184, p_value=8.015443640698077e-63)
dm_return(DM=16.988502246767563, p_value=2.085210794127189e-55)


In [44]:
# HS
print(DM_test(VT, VT_rw_1_hs, VT_rw_1_garch, len(VT_rw_1_hs)))
print(DM_test(VT, VT_rw_1_hs, VT_rw_1_earch, len(VT_rw_1_hs)))
print(DM_test(VT, VT_rw_1_hs, VT_rw_1_egarch, len(VT_rw_1_hs)))
print(DM_test(VT, VT_rw_1_hs, VT_rw_1_igarch, len(VT_rw_1_hs)))
print(DM_test(VT, VT_rw_1_hs, VT_rw_1_figarch, len(VT_rw_1_hs)))
print(DM_test(VT, VT_rw_1_hs, VT_rw_1_figarch0633, len(VT_rw_1_hs)))

dm_return(DM=38.38254722558919, p_value=9.80745447097009e-183)
dm_return(DM=45.981425368027324, p_value=1.5259626733688676e-225)
dm_return(DM=41.81143004360161, p_value=2.037020170298716e-202)
dm_return(DM=27.607440474623147, p_value=5.603566057179965e-118)
dm_return(DM=41.464184819103636, p_value=1.8868161685475228e-200)
dm_return(DM=39.140637297051484, p_value=3.8986261460376865e-187)


In [45]:
# BRW
print(DM_test(VT, VT_rw_1_brw, VT_rw_1_garch, len(VT_rw_1_brw)))
print(DM_test(VT, VT_rw_1_brw, VT_rw_1_earch, len(VT_rw_1_brw)))
print(DM_test(VT, VT_rw_1_brw, VT_rw_1_egarch, len(VT_rw_1_brw)))
print(DM_test(VT, VT_rw_1_brw, VT_rw_1_igarch, len(VT_rw_1_brw)))
print(DM_test(VT, VT_rw_1_brw, VT_rw_1_figarch, len(VT_rw_1_brw)))
print(DM_test(VT, VT_rw_1_brw, VT_rw_1_figarch0633, len(VT_rw_1_brw)))

dm_return(DM=1.374851597954961, p_value=0.169567262717638)
dm_return(DM=-10.612588610982733, p_value=1.0916025799284531e-24)
dm_return(DM=2.4790623996291736, p_value=0.013380473402029552)
dm_return(DM=1.6042601967041727, p_value=0.10905638617863904)
dm_return(DM=3.326471109997672, p_value=0.0009201668633483845)
dm_return(DM=2.9092832854167554, p_value=0.0037242694131959593)


In [46]:
# GARCH
print(DM_test(VT, VT_rw_1_garch, VT_rw_1_earch, len(VT_rw_1_garch)))
print(DM_test(VT, VT_rw_1_garch, VT_rw_1_egarch, len(VT_rw_1_garch)))
print(DM_test(VT, VT_rw_1_garch, VT_rw_1_igarch, len(VT_rw_1_garch)))
print(DM_test(VT, VT_rw_1_garch, VT_rw_1_figarch, len(VT_rw_1_garch)))
print(DM_test(VT, VT_rw_1_garch, VT_rw_1_figarch0633, len(VT_rw_1_garch)))

dm_return(DM=-10.35875022577906, p_value=1.1529689227868882e-23)
dm_return(DM=4.4224365030908075, p_value=1.112642508777099e-05)
dm_return(DM=1.2563106602757992, p_value=0.20937473303822945)
dm_return(DM=8.789351768334175, p_value=9.286065158356623e-18)
dm_return(DM=6.1513308965271465, p_value=1.2198830198060253e-09)


In [47]:
# EARCH
print(DM_test(VT, VT_rw_1_earch, VT_rw_1_egarch, len(VT_rw_1_earch)))
print(DM_test(VT, VT_rw_1_earch, VT_rw_1_igarch, len(VT_rw_1_earch)))
print(DM_test(VT, VT_rw_1_earch, VT_rw_1_figarch, len(VT_rw_1_earch)))
print(DM_test(VT, VT_rw_1_earch, VT_rw_1_figarch0633, len(VT_rw_1_earch)))

dm_return(DM=12.138404307353506, p_value=3.273810771689605e-31)
dm_return(DM=9.346562543666428, p_value=9.055332770612644e-20)
dm_return(DM=13.933307944800491, p_value=1.2852917510099461e-39)
dm_return(DM=13.420592119084226, p_value=3.839401774148526e-37)


In [48]:
# EGARCH
print(DM_test(VT, VT_rw_1_egarch, VT_rw_1_igarch, len(VT_rw_1_egarch)))
print(DM_test(VT, VT_rw_1_egarch, VT_rw_1_figarch, len(VT_rw_1_egarch)))
print(DM_test(VT, VT_rw_1_egarch, VT_rw_1_figarch0633, len(VT_rw_1_egarch)))

dm_return(DM=0.46395602348001913, p_value=0.6428070654893744)
dm_return(DM=4.429143789435893, p_value=1.0794130803708609e-05)
dm_return(DM=2.6183977121701956, p_value=0.009003604753130365)


In [49]:
# IGARCH
print(DM_test(VT, VT_rw_1_igarch, VT_rw_1_figarch,  len(VT_rw_1_igarch)))
print(DM_test(VT, VT_rw_1_igarch, VT_rw_1_figarch0633, len(VT_rw_1_igarch)))

dm_return(DM=0.41472871822104945, p_value=0.6784531008578597)
dm_return(DM=0.33486922408006403, p_value=0.737812630388089)


In [50]:
# FIGARCH
print(DM_test(VT, VT_rw_1_figarch, VT_rw_1_figarch0633, len(VT_rw_1_figarch)))

dm_return(DM=-0.9395472019545804, p_value=0.347737044081745)


## 5.00 %

In [51]:
VT_rw_5_dn = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/DN/5%25/VT_VaR_RW-DN-5%25.csv')
VT_rw_5_hs = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/HS/5%25/VT_VaR_RW-HS-5%25.csv')
VT_rw_5_brw = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/BRW/5%25/VT_VaR_RW-BRW-5%25.csv')
VT_rw_5_garch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/GARCH/5%25/VT_VaR_RW-G-5%25.csv')
VT_rw_5_earch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EARCH/5%25/VT_VaR_RW-EA-5%25.csv')
VT_rw_5_egarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/EGARCH/5%25/VT_VaR_RW-E-5%25.csv')
VT_rw_5_igarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/IGARCH/IGARCH-5%25-RW/VT_RW-I-5%25.csv')
VT_rw_5_figarch = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH/FIGARCH-5%25-RW/VT_RW-FI-5%25.csv')
VT_rw_5_figarch0633 = pd.read_csv('https://raw.githubusercontent.com/raphaelyt/thesis199.11/main/data/csv/FIGARCH0.633/FIGARCH0.633-5%25-RW/VT_RW-FI0.633-5%25.csv')

In [52]:
# DN
print(DM_test(VT, VT_rw_5_dn, VT_rw_5_garch, len(VT_rw_5_dn)))
print(DM_test(VT, VT_rw_5_dn, VT_rw_5_earch, len(VT_rw_5_dn)))
print(DM_test(VT, VT_rw_5_dn, VT_rw_5_egarch, len(VT_rw_5_dn)))
print(DM_test(VT, VT_rw_5_dn, VT_rw_5_igarch, len(VT_rw_5_dn)))
print(DM_test(VT, VT_rw_5_dn, VT_rw_5_figarch, len(VT_rw_5_dn)))
print(DM_test(VT, VT_rw_5_dn, VT_rw_5_figarch0633, len(VT_rw_5_dn)))

dm_return(DM=14.47211011607648, p_value=2.8102647299849842e-42)
dm_return(DM=10.499538822935616, p_value=3.1351877538895415e-24)
dm_return(DM=16.462868353017353, p_value=1.4082435485043095e-52)
dm_return(DM=10.428895007638026, p_value=6.036003695110546e-24)
dm_return(DM=16.64291747829314, p_value=1.5282074155917556e-53)
dm_return(DM=15.499491035503913, p_value=1.660912423630114e-47)


In [53]:
# HS
print(DM_test(VT, VT_rw_5_hs, VT_rw_5_garch, len(VT_rw_5_hs)))
print(DM_test(VT, VT_rw_5_hs, VT_rw_5_earch, len(VT_rw_5_hs)))
print(DM_test(VT, VT_rw_5_hs, VT_rw_5_egarch, len(VT_rw_5_hs)))
print(DM_test(VT, VT_rw_5_hs, VT_rw_5_igarch, len(VT_rw_5_hs)))
print(DM_test(VT, VT_rw_5_hs, VT_rw_5_figarch, len(VT_rw_5_hs)))
print(DM_test(VT, VT_rw_5_hs, VT_rw_5_figarch0633, len(VT_rw_5_hs)))

dm_return(DM=12.356699455271404, p_value=3.410545017283229e-32)
dm_return(DM=6.892410493802898, p_value=1.1221107231702077e-11)
dm_return(DM=14.220514549584703, p_value=4.992150078321963e-41)
dm_return(DM=8.95616410743174, p_value=2.377189809456078e-18)
dm_return(DM=14.47116108095253, p_value=2.8410850929110763e-42)
dm_return(DM=13.440150483260078, p_value=3.0965768164486953e-37)


In [54]:
# BRW
print(DM_test(VT, VT_rw_5_brw, VT_rw_5_garch, len(VT_rw_5_brw)))
print(DM_test(VT, VT_rw_5_brw, VT_rw_5_earch, len(VT_rw_5_brw)))
print(DM_test(VT, VT_rw_5_brw, VT_rw_5_egarch, len(VT_rw_5_brw)))
print(DM_test(VT, VT_rw_5_brw, VT_rw_5_igarch, len(VT_rw_5_brw)))
print(DM_test(VT, VT_rw_5_brw, VT_rw_5_figarch, len(VT_rw_5_brw)))
print(DM_test(VT, VT_rw_5_brw, VT_rw_5_figarch0633, len(VT_rw_5_brw)))

dm_return(DM=-4.159742845101054, p_value=3.535281809138137e-05)
dm_return(DM=-16.32377440701871, p_value=7.76814455863855e-52)
dm_return(DM=-3.817743403367702, p_value=0.0001452179918200558)
dm_return(DM=-1.7762439722234011, p_value=0.07607752982608439)
dm_return(DM=-2.7381090539929365, p_value=0.006318418921805291)
dm_return(DM=-2.6045152984578355, p_value=0.00937334467258839)


In [55]:
# GARCH
print(DM_test(VT, VT_rw_5_garch, VT_rw_5_earch, len(VT_rw_5_garch)))
print(DM_test(VT, VT_rw_5_garch, VT_rw_5_egarch, len(VT_rw_5_garch)))
print(DM_test(VT, VT_rw_5_garch, VT_rw_5_igarch, len(VT_rw_5_garch)))
print(DM_test(VT, VT_rw_5_garch, VT_rw_5_figarch, len(VT_rw_5_garch)))
print(DM_test(VT, VT_rw_5_garch, VT_rw_5_figarch0633, len(VT_rw_5_garch)))

dm_return(DM=-9.664138388755047, p_value=5.858316813355855e-21)
dm_return(DM=4.53687753144265, p_value=6.596395188804173e-06)
dm_return(DM=1.156815218916705, p_value=0.24769753728898541)
dm_return(DM=8.432483841385885, p_value=1.599010725269612e-16)
dm_return(DM=5.861237964273154, p_value=6.745245277819804e-09)


In [56]:
# EARCH
print(DM_test(VT, VT_rw_5_earch, VT_rw_5_egarch, len(VT_rw_5_earch)))
print(DM_test(VT, VT_rw_5_earch, VT_rw_5_igarch, len(VT_rw_5_earch)))
print(DM_test(VT, VT_rw_5_earch, VT_rw_5_figarch, len(VT_rw_5_earch)))
print(DM_test(VT, VT_rw_5_earch, VT_rw_5_figarch0633, len(VT_rw_5_earch)))

dm_return(DM=11.29380376633248, p_value=1.5925185026089588e-27)
dm_return(DM=8.676677782725534, p_value=2.3041920073792323e-17)
dm_return(DM=12.837793865057304, p_value=2.1259727136093883e-34)
dm_return(DM=12.352177261715028, p_value=3.575113769985179e-32)


In [57]:
# EGARCH
print(DM_test(VT, VT_rw_5_egarch, VT_rw_5_igarch, len(VT_rw_5_egarch)))
print(DM_test(VT, VT_rw_5_egarch, VT_rw_5_figarch, len(VT_rw_5_egarch)))
print(DM_test(VT, VT_rw_5_egarch, VT_rw_5_figarch0633, len(VT_rw_5_egarch)))

dm_return(DM=0.35380418861822177, p_value=0.7235800357397186)
dm_return(DM=4.061642702673577, p_value=5.35883548694826e-05)
dm_return(DM=2.3552416388234705, p_value=0.018754286499924288)


In [58]:
# IGARCH
print(DM_test(VT, VT_rw_5_igarch, VT_rw_5_figarch,  len(VT_rw_5_igarch)))
print(DM_test(VT, VT_rw_5_igarch, VT_rw_5_figarch0633, len(VT_rw_5_igarch)))

dm_return(DM=0.46657757347078394, p_value=0.6409307707479197)
dm_return(DM=0.386971302259817, p_value=0.6988816706501793)


In [59]:
# FIGARCH
print(DM_test(VT, VT_rw_5_figarch, VT_rw_5_figarch0633, len(VT_rw_5_figarch)))

dm_return(DM=-0.9867316683707175, p_value=0.3240763076538671)
